In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from chiragscosmology import Cosmology

In [3]:
path = '/global/homes/e/ecastori/PostBorn/'

In [4]:
data = np.loadtxt(path+'FFT_k-1Pk.dat',skiprows=2)

#----------------------#
n         = data[:,0]
Re_c_n    = data[:,1]
Im_c_n    = data[:,2]
Re_nu_n   = data[:,3]
Im_nu_n   = data[:,4]
#----------------------#

c_n  = Re_c_n  + 1j * Im_c_n
nu_n = Re_nu_n + 1j * Im_nu_n

In [5]:
data = np.loadtxt(path+'Il_nu_t.dat',skiprows=1)

#----------------------#
ell       = data[:,0]
nn        = data[:,2]
tt        = data[:,1]
Re_I      = data[:,3]
Im_I      = data[:,4]
#----------------------#

I_ = Re_I + 1j * Im_I

In [6]:
#load class output
data = np.loadtxt('class_ouput.out')
ll_  = data[0]
clpp = data[1]
clpp2= data[2]

# load results from Valentins paper
data=np.loadtxt(path+'Cell_marko.dat')
cl = data[:,1]
ls = data[:,0]

In [9]:
print(ll_)
print(clpp[10])

[   0.    1.    2. ...  998.  999. 1000.]
5.516622122583256e-11


In [ ]:
#cosmology
h         = 0.6770
omega_cdm = 0.11923
omega_b   = 0.02247
Omega_b   = omega_b/h**2
Omega_cdm = omega_cdm/h**2
Omega_m   = Omega_b+Omega_cdm
ns        = 0.96824
A0        = 2.10732*10**(-9)
sigma_8   = 0.8221362793660904 #computed from CLASS

#dictionary for class
cosmo_dict={'h': h,
'omega_b' : omega_b,
'omega_cdm': omega_cdm,
'A_s'    : A0,
'n_s'    : ns,
'k_pivot' : 0.05}
print(cosmo_dict)

# speed of light
c      = 299792458./1000. # km/s

#prefactor for Cl_kk computation from Cl_dd
prefac = 1.5*Omega_m*(100.)**2/c**2 #without h

In [ ]:
#get comoving distance from integrating c/H_0/E(z)
def integrand(z):
    return c/(100.)*(Omega_m*(1.+z)**3+(1.-Omega_m))**(-0.5)#Mpc/h

def chi(z):
    if z==0:
        return 0
    else:
        if z<1.:
            z_ = np.arange(0.,z,1e-6)
        else:
            z_ = np.arange(0.,z,1e-4)
        res = np.trapz(integrand(z_),z_)
        return res

z_cmb = 1100

In [ ]:
from scipy.interpolate import InterpolatedUnivariateSpline as ius

z_= np.logspace(-4,np.log10(z_cmb))

chi_z_=[]
for zz in z_:
    print(zz)
    chi_z_+=[chi(zz)]

# chi_cmb
chi_cmb = chi(z_cmb)
chi_z   = ius(z_,chi_z_)
z_chi   = ius(chi_z_,z_)

In [ ]:
# sanitary check plots
z=np.linspace(0.,50,200)
plt.figure()
plt.plot(z,chi_z(z))
plt.show()
plt.figure()
plt.plot(z,z_chi(chi_z(z)))
plt.show()

In [ ]:
#initliaze Chirags cosmology
cosmo = Cosmology(M=Omega_m,B=Omega_b,sig8=sigma_8)

In [ ]:
def set_kernel(chi_max):
    def kernel(chi):
        z = z_chi(chi)
        return (1.+z)*cosmo.Dgrow(z)
    return kernel

In [ ]:
W = set_kernel(chi_cmb)

In [ ]:
# check if everything was read in correctly
ell_ = np.unique(ell)
nu_n_= np.unique(nu_n) 
print(nu_n_)
print(ell_)
print(np.unique(tt))
t_=np.unique(tt)
print(len(nu_n_),len(n),len(c_n),len(t_))
print(len(tt),len(I_))

In [ ]:
# precompute sum over n for reuse

nu=nu_n_
res=[]
m = 0
for ii,ll in enumerate(ell_):
    print(ii,ll)
    for t in t_:
        for r in t_:
            res+=[np.real(sum(I_[m:m+len(n)][0:-1]*c_n[0:-1]*2.*(chi_cmb*r)**(-nu[0:-1]))+I_[m:m+len(n)][-1]*c_n[-1]*(chi_cmb*r)**(-nu[-1]))]
        m+=len(n)
assert(m==len(I_))

res=np.asarray(res)


In [ ]:
Cls=[]
labels=[]

In [ ]:
# this is without interpolation and all kernels in the inner integral
from scipy.integrate import simps, quadrature
from scipy.interpolate import interp1d
m=0
result=[]
plt.figure()
for ii,ll in enumerate(ell_):
    res_=[]
    for t in t_:
        
        #for r in t_:
        integrand=W(t*t_*chi_cmb)*W(t*chi_cmb)*(1.-t_*t)/t*(1.-t_)/t_*res[m:m+len(t_)]

            
        m+=len(t_)
        integrand[0] = 0.
        if ll==3:
            plt.semilogx(t_,integrand)
        res_+=[simps(integrand,t_)]
    integrand = np.asarray(res_)

    
    integrand[0] = 0.
    result+=[simps(integrand,t_)]
plt.show()

Cl=np.real(np.asarray(result))*4./np.pi**2*prefac**2

Cls+=[Cl]
labels+=['no CLASS, no interp']

plt.figure()#
plt.semilogx(ell_,Cl/np.interp(ell_,ll_,clpp), label='our result over CLASS no Limber')
plt.semilogx(ell_,Cl/np.interp(ell_,ll_,clpp)/1.13, label='our result/1.13 over CLASS no Limber')
#plt.semilogx(ell_,np.interp(ell_,ls,cl)/Cl)
plt.semilogx(ls,cl/np.interp(ls,ll_,clpp), label='Markos result over CLASS no Limber')
plt.semilogx(ll_,clpp2/clpp, label='CLASS Limber over CLASS no Limber') 
#plt.axhline(1,color='black')
plt.ylim(0.75,1.5)
plt.grid()
plt.legend()
plt.show()

In [ ]:
# linear interpolation
from scipy.integrate import simps, quadrature
from scipy.interpolate import interp1d
m=0
result=[]
plt.figure()
for ii,ll in enumerate(ell_):
    res_=[]
    for t in t_:
        
        #for r in t_:
        integrand=W(t*t_*chi_cmb)*W(t*chi_cmb)*(1.-t_*t)/t*(1.-t_)/t_*res[m:m+len(t_)]

            
        m+=len(t_)
        integrand[0] = 0.#+1j*0.

        integrand = interp1d(t_,integrand,kind='linear')
        t_long  = np.linspace(min(t_),max(t_),500)
        if ll==3:
            plt.semilogx(t_long,integrand(t_long))
        res_+=[simps(integrand(t_long),t_long)]#quadrature(integrand,min(t_),max(t_))[0]]
    #print(res_)
        # for every n do r integral
    integrand = np.asarray(res_)

    
    integrand[0] = 0.#+1j*0.
    integrand = interp1d(t_,integrand,kind='linear')
    result+=[simps(integrand(t_long),t_long)]#min(t_),max(t_))[0]]
plt.show()    
Cl=np.real(np.asarray(result))*4./np.pi**2*prefac**2

Cls+=[Cl]
labels+=['no CLASS, lin interp']

plt.figure()#
plt.semilogx(ell_,Cl/np.interp(ell_,ll_,clpp), label='our result over CLASS no Limber')
plt.semilogx(ell_,Cl/np.interp(ell_,ll_,clpp)/1.13, label='our result/1.13 over CLASS no Limber')
#plt.semilogx(ell_,np.interp(ell_,ls,cl)/Cl)
plt.semilogx(ls,cl/np.interp(ls,ll_,clpp), label='Markos result over CLASS no Limber')
plt.semilogx(ll_,clpp2/clpp, label='CLASS Limber over CLASS no Limber') 
#plt.axhline(1,color='black')
plt.ylim(0.75,1.5)
plt.grid()
plt.legend()
plt.show()

In [ ]:
# cubic interpolation
from scipy.integrate import simps, quadrature
from scipy.interpolate import interp1d
m=0
result=[]
plt.figure()
for ii,ll in enumerate(ell_):
    res_=[]
    for t in t_:
        
        #for r in t_:
        integrand=W(t*t_*chi_cmb)*W(t*chi_cmb)*(1.-t_*t)/t*(1.-t_)/t_*res[m:m+len(t_)]

            
        m+=len(t_)
        integrand[0] = 0.#+1j*0.

        integrand = interp1d(t_,integrand,kind='cubic')
        t_long  = np.linspace(min(t_),max(t_),500)
        if ll==3:
            plt.semilogx(t_long,integrand(t_long))
        res_+=[simps(integrand(t_long),t_long)]#quadrature(integrand,min(t_),max(t_))[0]]
    #print(res_)
        # for every n do r integral
    integrand = np.asarray(res_)

    
    integrand[0] = 0.#+1j*0.
    integrand = interp1d(t_,integrand,kind='cubic')
    result+=[simps(integrand(t_long),t_long)]#min(t_),max(t_))[0]]
plt.show()
Cl=np.real(np.asarray(result))*4./np.pi**2*prefac**2

Cls+=[Cl]
labels+=['no CLASS, cubic interp']

plt.figure()#
plt.semilogx(ell_,Cl/np.interp(ell_,ll_,clpp), label='our result over CLASS no Limber')
plt.semilogx(ell_,Cl/np.interp(ell_,ll_,clpp)/1.13, label='our result/1.13 over CLASS no Limber')
#plt.semilogx(ell_,np.interp(ell_,ls,cl)/Cl)
plt.semilogx(ls,cl/np.interp(ls,ll_,clpp), label='Markos result over CLASS no Limber')
plt.semilogx(ll_,clpp2/clpp, label='CLASS Limber over CLASS no Limber') 
#plt.axhline(1,color='black')
plt.ylim(0.75,1.5)
plt.grid()
plt.legend()
plt.show()

In [ ]:
# CLASS version 
from classy import Class
cosmo = Class()
cosmo.set(cosmo_dict)
cosmo.compute()
cosmo_b               = cosmo.get_background()

class_z               = cosmo_b['z'][::-1]
class_chi             = cosmo_b['comov. dist.'][::-1]
class_D               = cosmo_b['gr.fac. D'][::-1]#/cosmo_b['gr.fac. D'][-1]

derivParams           = cosmo.get_current_derived_parameters(['z_rec'])
z_cmb                 = derivParams['z_rec']


from scipy.interpolate import interp1d
from scipy.interpolate import InterpolatedUnivariateSpline as ius
chi_z = interp1d(class_z,class_chi*h)
z_chi = interp1d(class_chi*h,class_z)  # Mpc/h
D_chi = interp1d(class_chi*h,class_D)    # growth
D_z   = interp1d(class_z,class_D)
# chi_cmb
chi_cmb = chi_z(z_cmb)


def set_kernel(chi_max):
    def kernel(chi):
        z = z_chi(chi)
        return (1.+z)*D_chi(chi)
    return kernel

W_CLASS = set_kernel(chi_cmb)

In [ ]:
# computation with CLASS derived distances and growth
from scipy.integrate import simps, quadrature
from scipy.interpolate import interp1d
m=0
result=[]
plt.figure()
for ii,ll in enumerate(ell_):
    res_=[]
    for t in t_:
        
        #for r in t_:
        integrand=W_CLASS(t*t_*chi_cmb)*W_CLASS(t*chi_cmb)*(1.-t_*t)/t*(1.-t_)/t_*res[m:m+len(t_)]

            
        m+=len(t_)
        integrand[0] = 0.
        integrand = interp1d(t_,integrand,kind='cubic')
        t_long  = np.linspace(min(t_),max(t_),500)
        if ll==3:
            plt.semilogx(t_long,integrand(t_long))
        res_+=[simps(integrand(t_long),t_long)]
    integrand = np.asarray(res_)
    
    integrand[0] = 0.
    integrand = interp1d(t_,integrand,kind='cubic')
    result+=[simps(integrand(t_long),t_long)]
plt.show()

Cl=np.real(np.asarray(result))*4./np.pi**2*prefac**2
Cls+=[Cl]
labels+=['CLASS, cubic interp']

plt.figure()#
plt.semilogx(ell_,Cl/np.interp(ell_,ll_,clpp), label='our result over CLASS no Limber')
plt.semilogx(ell_,Cl/np.interp(ell_,ll_,clpp)/1.13, label='our result/1.13 over CLASS no Limber')
#plt.semilogx(ell_,np.interp(ell_,ls,cl)/Cl)
plt.semilogx(ls,cl/np.interp(ls,ll_,clpp), label='Markos result over CLASS no Limber')
plt.semilogx(ll_,clpp2/clpp, label='CLASS Limber over CLASS no Limber') 
#plt.axhline(1,color='black')
plt.ylim(0.75,1.5)
plt.grid()
plt.legend()
plt.show()

In [ ]:
# computation with CLASS derived distances and growth
from scipy.integrate import simps, quadrature
from scipy.interpolate import interp1d
m=0
result=[]
plt.figure()
for ii,ll in enumerate(ell_):
    res_=[]
    for t in t_:
        
        #for r in t_:
        integrand=W_CLASS(t*t_*chi_cmb)*W_CLASS(t*chi_cmb)*(1.-t_*t)/t*(1.-t_)/t_*res[m:m+len(t_)]

            
        m+=len(t_)
        integrand[0] = 0.
        if ll==3:
            plt.semilogx(t_,integrand)
        res_+=[simps(integrand,t_)]
    integrand = np.asarray(res_)

    
    integrand[0] = 0.
    result+=[simps(integrand,t_)]
plt.show()

Cl=np.real(np.asarray(result))*4./np.pi**2*prefac**2
Cls+=[Cl]
labels+=['CLASS, no interp']

plt.figure()#
plt.semilogx(ell_,Cl/np.interp(ell_,ll_,clpp), label='our result over CLASS no Limber')
plt.semilogx(ell_,Cl/np.interp(ell_,ll_,clpp)/1.13, label='our result/1.13 over CLASS no Limber')
#plt.semilogx(ell_,np.interp(ell_,ls,cl)/Cl)
plt.semilogx(ls,cl/np.interp(ls,ll_,clpp), label='Markos result over CLASS no Limber')
plt.semilogx(ll_,clpp2/clpp, label='CLASS Limber over CLASS no Limber') 
#plt.axhline(1,color='black')
plt.ylim(0.75,1.5)
plt.grid()
plt.legend()
plt.show()

In [ ]:
cl = np.interp(ell_,ll_,clpp)
plt.figure()
for ii,Cl in enumerate(Cls):
    plt.semilogx(ell_,Cl/cl, label=labels[ii])
    plt.grid()
plt.ylabel('$C_L^{\kappa\kappa}/CLASS$')
plt.ylim(0.75,2.)
plt.legend()
plt.show()

In [ ]:
c_ell,cl_limber,cl_limber2,cl_camb = np.loadtxt('camb_cls.out')
cl = np.interp(ell_,c_ell,cl_camb*2.*np.pi/(c_ell*(c_ell+1))**2)
plt.figure()
for ii,Cl in enumerate(Cls):
    plt.semilogx(ell_,Cl/cl, label=labels[ii])
    plt.grid()
plt.ylabel('$C_L^{\kappa\kappa}/CAMB$')
plt.ylim(0.75,2.)
plt.legend()
plt.show()